In [1]:
import cv2
import numpy as np
import mediapipe as mp

In [2]:
def list_available_cameras(max_index):
    """
    Lists the available camera indices.

    Args:
        max_index (int): The maximum index to check for available cameras.

    Returns:
        list: A list of indices where cameras are available.
    """

    # Store the indices of available cameras
    available_cameras = []

    # Iterate through the indices
    for i in range(max_index):
        cap = cv2.VideoCapture(i)

        # Check if the camera is available
        if cap.isOpened():
            # Add the index to the list of available cameras
            available_cameras.append(i)
            cap.release()        

    return available_cameras

max_camera_index = 10 # Maximum index to check for available cameras
available_cameras = list_available_cameras(max_camera_index) # Call the function

[ WARN:0@1.525] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@1.623] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@1.710] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video3): can't open camera by index
[ERROR:0@1.712] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@1.712] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video4): can't open camera by index
[ERROR:0@1.713] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@1.713] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video5): can't open camera by index
[ERROR:0@1.713] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@1.714] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video6): can't open camera by index
[ERROR:0@1.714] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup C

In [3]:
# Print the available cameras

print("Available Cameras:", available_cameras)

Available Cameras: [0, 2]


In [ ]:
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

image = cv2.imread('Putin.jpg')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

results = face_detection.process(image_rgb)
if not results.detections:
    print("No face detected in the image.")
    exit()

face_image = None
for detection in results.detections:
    bboxC = detection.location_data.relative_bounding_box
    ih, iw, _ = image.shape
    (x, y, w, h) = (int(bboxC.xmin * iw), int(bboxC.ymin * ih),
                    int(bboxC.width * iw), int(bboxC.height * ih))
    face_image = image[y:y+h, x:x+w]
    break

cap = cv2.VideoCapture(0)

with mp_face_detection.FaceDetection(
        model_selection=1, min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(frame_rgb)

        if results.detections:
            for detection in results.detections:
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = frame.shape
                (x, y, w, h) = (int(bboxC.xmin * iw), int(bboxC.ymin * ih),
                                int(bboxC.width * iw), int(bboxC.height * ih))

                resized_face_image = cv2.resize(face_image, (w, h))
                
                frame[y:y+h, x:x+w] = resized_face_image

        cv2.imshow('Face Swap', frame)
        
        if cv2.waitKey(0) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()